In [ ]:
pip install spacy; torch

In [2]:
import pandas as pd
import torch

In [3]:
ends = pd.read_csv("/Users/brentkong/Documents/curling/data/Ends.csv")
games = pd.read_csv("/Users/brentkong/Documents/curling/data/Games.csv")
stones = pd.read_csv("/Users/brentkong/Documents/curling/data/Stones.csv")

In [4]:
games["GameUID"] = games.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
ends["GameUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
ends["EndUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}_{int(r.EndID)}", axis=1)
stones["GameUID"] = stones.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
stones["EndUID"] = stones.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}_{int(r.EndID)}", axis=1)

In [5]:
def assign_hammer(row):
    if row.ShotID % 2 != 0:
        return 0
    else:
        return 1
    return None

stones["Has_Hammer"] = stones.apply(assign_hammer, axis = 1)


In [6]:
ends = ends.merge(games[["GameUID","NOC1","NOC2","TeamID1","TeamID2","LSFE"]], on="GameUID", how="left")

def get_opp_noc(row):
    if row.TeamID == row.TeamID1:
        return row.NOC2
    if row.TeamID == row.TeamID2:
        return row.NOC1
    return None

ends["OpponentNOC"] = ends.apply(get_opp_noc, axis=1)
ends = ends.merge(stones[["EndUID", "Has_Hammer"]], on="EndUID", how="left").drop_duplicates().reset_index(drop = True)

In [7]:
def compute_geometry(board_x, board_y):
    house = [(x,y) for x,y in zip(board_x,board_y) if 0 < x < 1500 and 0 < y < 3000] # no 0 or 4095
    if not house:
        return 0,0,0,0

    # Burial depth: how close a house stone is to center
    cx, cy = 750, 800
    burial = min(((x-cx)**2 + (y-cy)**2)**0.5 for x,y in house)

    # Guard cover angle: angle between closest stone and center
    gx, gy = house[0]
    angle = torch.atan2(torch.tensor(gy-cy), torch.tensor(gx-cx)).item()

    # Clustering: how tight stones are packed
    cluster = sum(((x-cx)**2 + (y-cy)**2)**0.5 < 300 for x,y in house) / len(house)

    # Side openness: difference between left and right free space
    left_open = sum(x < cx for x,_ in house)
    right_open = sum(x > cx for x,_ in house)
    openness = right_open - left_open

    return burial, angle, cluster, openness

stones = stones.copy()
stones["BurialDepth"], stones["GuardAngle"], stones["ClusterIndex"], stones["SideOpenness"] = zip(
    *stones.apply(lambda r: compute_geometry(
        [r[f"stone_{i}_x"] for i in range(1,13)],
        [r[f"stone_{i}_y"] for i in range(1,13)]
    ), axis=1)
)

In [8]:
tendency = (
    ends.groupby("OpponentNOC")["PowerPlay"]
    .value_counts(normalize=True)
    .unstack(fill_value=0)   # Only 1 and 2 become columns
)

tendency.columns = ["PP_Right", "PP_Left"]
ends = ends.merge(tendency, on="OpponentNOC", how="left")

In [9]:
geom = stones.groupby("EndUID")[["BurialDepth","GuardAngle","ClusterIndex","SideOpenness"]].mean()
ends = ends.merge(geom, on="EndUID", how="left").fillna(0)

In [10]:
# Final modeling table
ends_df = ends[[
    "GameUID", "EndID", "TeamID", "PowerPlay", "Result", "Has_Hammer", "OpponentNOC",
    "PP_Right", "PP_Left","BurialDepth","GuardAngle","ClusterIndex","SideOpenness"
]]

games_df = games[[
    "GameUID", "NOC1",	"NOC2",	"ResultStr1", "ResultStr2",	"LSFE",	"Winner",
    "TeamID1", "TeamID2",
]]


stones_df = stones[[
    'GameUID', 'EndID', 'ShotID', 'TeamID', 'PlayerID', 'Task', 'Handle', 'Points', 'stone_1_x',
    'stone_1_y', 'stone_2_x', 'stone_2_y', 'stone_3_x', 'stone_3_y',
    'stone_4_x', 'stone_4_y', 'stone_5_x', 'stone_5_y', 'stone_6_x',
    'stone_6_y', 'stone_7_x', 'stone_7_y', 'stone_8_x', 'stone_8_y',
    'stone_9_x', 'stone_9_y', 'stone_10_x', 'stone_10_y', 'stone_11_x',
    'stone_11_y', 'stone_12_x', 'stone_12_y',
    'Has_Hammer', 'BurialDepth', 'GuardAngle', 'ClusterIndex',
    'SideOpenness'
]]

ends_df.to_csv("/Users/brentkong/Documents/curling/processed_data/ends_processed.csv", index=False)
games_df.to_csv("/Users/brentkong/Documents/curling/processed_data/games_processed.csv", index=False)
stones_df.to_csv("/Users/brentkong/Documents/curling/processed_data/stones_processed.csv", index=False)